# MySQL 접속용 샘플 프로그램
- pip install pymysql

### 1. 접속 정보가 들어있는 파일 읽기

In [2]:
import json
with open('./mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'boot_user',
 'password': 'boot_pass',
 'database': 'world',
 'port': 3306}

### 2. MySQL에 접속

In [ ]:
import pymysql
conn = pymysql.connect(
	host = config['host'],
	user = config['user'],
	password = config['password'],
	database = config['database'],
	port = config['port']
)
# conn = pymysql.connect(**config)

### 3. MySQL Access
- city 테이블에서 인구가 900만이 넘는 도시 정보

In [ ]:
cur = conn.cursor()
sql = 'select * from city where population >= 9000000 order by population desc'
cur.execute(sql)

rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)

(1024, 'Mumbai (Bombay)', 'IND', 'Maharashtra', 10500000)
(2331, 'Seoul', 'KOR', 'Seoul', 9981619)
(206, 'SÃ£o Paulo', 'BRA', 'SÃ£o Paulo', 9968485)
(1890, 'Shanghai', 'CHN', 'Shanghai', 9696300)
(939, 'Jakarta', 'IDN', 'Jakarta Raya', 9604900)
(2822, 'Karachi', 'PAK', 'Sindh', 9269265)


In [7]:
import pandas as pd
df = pd.DataFrame(rows, columns=['id','name','country_code','district','population'])
df.set_index('id', inplace=True)
df

,name,country_code,district,population
id,,,,
1024,Mumbai (Bombay),IND,Maharashtra,10500000
2331,Seoul,KOR,Seoul,9981619
206,SÃ£o Paulo,BRA,SÃ£o Paulo,9968485
1890,Shanghai,CHN,Shanghai,9696300
939,Jakarta,IDN,Jakarta Raya,9604900
2822,Karachi,PAK,Sindh,9269265


- 검색: 국가별 인구수 Top 5

In [10]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE countrycode=%s ORDER BY population DESC LIMIT 5"
cur.execute(sql, ('KOR',))		# 파라메터는 반드시 튜플로 전달해야 함
rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 9981619)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)


- 검색: name field로 검색 --> 1건의 결과

In [11]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE name=%s"
cur.execute(sql, ('Seoul',))
row = cur.fetchone()
cur.close()
row

(2331, 'Seoul', 'KOR', 'Seoul', 9981619)

- 데이터 추가(Insert)

In [12]:
cur = conn.cursor()
sql = "INSERT INTO koreancity VALUES(DEFAULT, '도시', '광역시도', 300000)"
cur.execute(sql)

1

In [13]:
cur = conn.cursor()
sql = "select * from koreancity where name='도시'"
cur.execute(sql)
row = cur.fetchone()
print(row)

(2408, '도시', '광역시도', 300000)


In [14]:
conn.commit()		# DB에 반영(Disk), lock이 해제가 되어서 다른 사람이 읽고 쓰기 할 수 있음
cur.close()

In [15]:
params = ('도시2', '광역시도2', 400000)

cur = conn.cursor()
sql = "INSERT INTO koreancity VALUES(DEFAULT, %s, %s, %s)"
cur.execute(sql, params)
conn.commit()
cur.close()

In [16]:
params_list = [
	('도시3', '광역시도3', 400003),
	('도시4', '광역시도4', 400004)
]

cur = conn.cursor()
sql = "INSERT INTO koreancity VALUES(DEFAULT, %s, %s, %s)"
# for params in params_list:
# 	cur.execute(sql, params)
cur.executemany(sql, params_list)
conn.commit()
cur.close()

- 데이터 수정(Update)

In [17]:
sql = "UPDATE koreancity SET district='강원' WHERE id = 2409"
cur = conn.cursor()
cur.execute(sql)
conn.commit()
cur.close()

In [ ]:
params_list = [('경기', 2410), ('경기', 2411)]
sql = "UPDATE koreancity SET district=%s WHERE id = %s"
cur = conn.cursor()
cur.executemany(sql, params_list)
conn.commit()
cur.close()

- 데이터 삭제(Delete)

In [19]:
sql = "delete from koreancity where id >= %s"
cur = conn.cursor()
cur.execute(sql, (2408, ))
conn.commit()
cur.close()

### 4. 접속 해제

In [20]:
conn.close()

### 5. DAO(Data Access Object)를 만들어서 처리하는 것을 권장
- CRUD 기본
- id로 검색 - 한건의 결과 - tuple로 반환
- 모든 데이터 검색 - 여러건의 결과 - tuple의 리스트로 반환
- Insert - row 데이터 전체를 tuple로
- Update - row 데이터 전체를 tuple로
- Delete - id 값을 tuple로